In [ ]:
!nvidia-smi

Mon May 17 17:13:24 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip3 install -U gitpython
!pip3 install transformers
!pip3 install urllib3

Requirement already up-to-date: gitpython in /usr/local/lib/python3.7/dist-packages (3.1.17)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd 
from tqdm import tqdm
from tqdm import trange
import gc
import copy
import os
import re
import string
import random
import math
import time
import datetime

from collections import Counter
import itertools
#from nltk.tokenize import word_tokenize
#from nltk.corpus import stopwords
#import nltk
#nltk.download('stopwords')
#nltk.download('punkt')

#import matplotlib.pyplot as plt
#import matplotlib.ticker as ticker
#import matplotlib.dates as mdates
#import seaborn as sns
#plt.style.use('bmh')

import torch
from transformers import AutoModelWithLMHead, AutoTokenizer, GPT2Tokenizer, GPT2LMHeadModel, GPT2Config, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, random_split, DataLoader, RandomSampler, SequentialSampler
from torch.nn import functional as F

pd.set_option('display.max_rows', 50)

In [ ]:
device = torch.device('cuda')

In [ ]:
data_path = "./drive/MyDrive/anna/data/accentedPoems.csv"

In [ ]:
data = pd.read_csv(data_path)

In [ ]:
data.fillna("", inplace=True)

In [ ]:
data

,text
0,В первом до̀ме ро̀бко и до̀лго жѝл я в дѐтст...
1,"Кака̀я химѐра\nмой ра̀зум имѐла,\nмой ра̀зум..."
2,"Кто пьѐт одѝн, тот пьѐт с умо̀м\nс умо̀м св..."
3,Не вѝдно о̀строу̀мных ра̀сстава̀ний\nво тьмѐ...
4,"Ни ша̀тко ни ва̀лко гудѝт кофева̀рка,\nгудѝ,..."
...,...
54183,Читателю\n\nПервы труд мой в французском приим...
54184,Элегия о смерти Петра Великого\n\nЧто за печал...
54185,СТЯЖАТЕЛЯ СИХ КНИГ ПОСЛЕДНЕЕ КНИГАМ ЦЕЛОВАНИЕ\...
54186,"О титулы, пропасти паче вас назва̀ти,\nКоль вы..."


In [ ]:
def preprocess(df):
    new_texts = []
    texts = df['text'].values
    for i in range(len(texts)):
        texts[i] = texts[i].lower()
        texts[i] = re.sub("[0-9]", "", texts[i])
        texts[i] = re.sub(' +', ' ', texts[i])
        if len(texts[i]) >= 100:
            new_texts.append(texts[i])
    print(len(new_texts))
    new_df = pd.DataFrame(data=new_texts, columns=['text'])
    return new_df

data = preprocess(data)
data

53579


,text
0,в первом до̀ме ро̀бко и до̀лго жѝл я в дѐтст...
1,"кака̀я химѐра\nмой ра̀зум имѐла,\nмой ра̀зум..."
2,"кто пьѐт одѝн, тот пьѐт с умо̀м\nс умо̀м св..."
3,не вѝдно о̀строу̀мных ра̀сстава̀ний\nво тьмѐ...
4,"ни ша̀тко ни ва̀лко гудѝт кофева̀рка,\nгудѝ,..."
...,...
53574,читателю\n\nпервы труд мой в французском приим...
53575,элегия о смерти петра великого\n\nчто за печал...
53576,стяжателя сих книг последнее книгам целование\...
53577,"о титулы, пропасти паче вас назва̀ти,\nколь вы..."


In [ ]:
model = AutoModelWithLMHead.from_pretrained('sberbank-ai/rugpt3large_based_on_gpt2')

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:810: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [ ]:
model = model.to(device)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('sberbank-ai/rugpt3large_based_on_gpt2')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### **ОБУЧЕНИЕ**

In [ ]:
raw = data['text'].to_list()

In [ ]:
tokens = [tokenizer.encode(text) for text in raw]

In [ ]:
validation_start_index = int(len(raw) * 0.1)
validation_start_index

5357

In [ ]:
train_lengths = [len(token) for token in tqdm(tokens)]

100%|██████████| 53579/53579 [00:00<00:00, 2343709.80it/s]


In [ ]:
print(tokenizer.vocab_size)

50257


In [ ]:
batch_size = 1
random_seed = 420
torch.manual_seed(random_seed)
max_len = 100

In [ ]:
special_tokens_dict = {'pad_token': '<PAD>', 'bos_token': '<BOS>', 'eos_token': '<EOS>'}

In [ ]:
pad_index = 50258
boc_index = 50259
eoc_index = 50260

In [ ]:
num_added_tokens = tokenizer.add_special_tokens(special_tokens_dict)

In [ ]:
class PoetryDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length):
        self.tokenizer = tokenizer
        self.texts = texts
        self.max_len = max_length

    def __len__(self):        
        return len(self.texts)
    
    def __getitem__(self, index):

        text = self.texts[index]
        text = f'{text}'
        
        enc_dict = tokenizer(text, truncation=True, max_length=self.max_len)
        tokenized = enc_dict['input_ids']
        
        return torch.tensor(tokenized).long()

In [ ]:
poems_train = PoetryDataset(raw, tokenizer, max_len)
poems_valid = PoetryDataset(raw[::10], tokenizer, max_len)

len(poems_train), len(poems_valid)

(53579, 5358)

In [ ]:
train_loader = DataLoader(poems_train,
                          batch_size=batch_size, 
                          shuffle=True)

validation_loader = DataLoader(poems_valid, 
                               batch_size=batch_size, 
                               shuffle=True)

In [ ]:
num_epochs = 1

In [ ]:
learning_rate = 0.0001
warmup_steps = 50
total_steps = len(train_loader) * num_epochs

In [ ]:
model.resize_token_embeddings(len(tokenizer))

Embedding(50261, 1536)

In [ ]:
optimizer = AdamW(model.parameters(), lr=learning_rate)

In [ ]:
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=warmup_steps,
                                            num_training_steps=total_steps)

In [ ]:
def train(model, loader, optimizer, scheduler, last_n_losses=200, verbose=True):

    losses = []

    progress_bar = tqdm(total=len(loader.dataset), disable=not verbose, desc='Train')

    model.train()

    for tokens in loader:

        tokens = tokens.to(device)
        
        outputs = model(tokens)

        shift_tokens = tokens[..., 0:].contiguous()
        shift_logits = outputs.logits[..., :, :].contiguous()

        losses_vec = F.cross_entropy(shift_logits.view(-1, shift_logits.size(-1)), shift_tokens.view(-1), reduction="none")

        coeffs = torch.ones(list(losses_vec.size())[0]).cuda()
        indexes = (shift_tokens == 206).nonzero(as_tuple=True)[1]
        for index in indexes:
            if index > 1:
                coeffs[index - 1] *= 3
                coeffs[index - 2] *= 3

        losses_vec = coeffs*losses_vec

        loss = losses_vec.mean()
        
        losses.append(loss.item())

        loss.backward()
        optimizer.zero_grad()
        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix(loss=np.mean(losses[-last_n_losses:]),
                                 perplexity=np.exp(np.mean(losses[-last_n_losses:])))
        
        progress_bar.update()

    progress_bar.close()
    
    return losses

In [ ]:
def evaluate(model, loader, last_n_losses=200, verbose=True):

    losses = []

    progress_bar = tqdm(total=len(loader), disable=not verbose, desc='Evaluate')

    model.eval()

    for tokens in loader:

        tokens = tokens.to(device)
        
        outputs = model(tokens)

        shift_tokens = tokens[..., 0:].contiguous()
        shift_logits = outputs.logits[..., :, :].contiguous()

        losses_vec = F.cross_entropy(shift_logits.view(-1, shift_logits.size(-1)), shift_tokens.view(-1), reduction="none")

        coeffs = torch.ones(list(losses_vec.size())[0]).cuda()
        indexes = (shift_tokens == 206).nonzero(as_tuple=True)[1]
        for index in indexes:
            if index > 1:
                coeffs[index - 1] *= 3
                coeffs[index - 2] *= 3

        losses_vec = coeffs*losses_vec

        loss = losses_vec.mean()
        
        losses.append(loss.item())

        progress_bar.set_postfix(loss=np.mean(losses[-last_n_losses:]),
                                 perplexity=np.exp(np.mean(losses[-last_n_losses:])))
        
        progress_bar.update()

    progress_bar.close()
    
    return losses

In [ ]:
save_best_model_path = './drive/MyDrive/anna/rhyme_best_model_state_dict.pth'
save_best_optimizer_path = './drive/MyDrive/anna/rhyme_best_optimizer_state_dict.pth'
save_last_model_path = './drive/MyDrive/anna/rhyme_last_model_state_dict.pth'
save_last_optimizer_path = './drive/MyDrive/anna/rhyme_last_optimizer_state_dict.pth'

In [ ]:
# ЧИСТИМ ПАМЯТЬ ПЕРЕД ОБУЧЕНИЕМ

gc.collect()
torch.cuda.empty_cache()
model = model.to(device)

In [ ]:
train_losses = []
validation_losses = []

train_perplexities = []
validation_perplexities = []

best_validation_loss = 1e+6

for n_epoch in range(1, num_epochs + 1):
    
    epoch_train_losses = train(model, train_loader, optimizer, scheduler)

    torch.save(model.state_dict(), save_last_model_path)
    torch.save(optimizer.state_dict(), save_last_optimizer_path)

    epoch_validation_losses = evaluate(model, validation_loader)
    
    mean_train_loss = np.mean(epoch_train_losses)
    mean_validation_loss = np.mean(epoch_validation_losses)
    
    train_losses.extend(epoch_train_losses)
    train_perplexities.append(np.exp(mean_train_loss))
    
    validation_losses.extend(epoch_validation_losses)
    validation_perplexities.append(np.exp(mean_validation_loss))
    
    message = f'Epoch: {n_epoch}\n'
    message += f'Train: loss - {mean_train_loss:.4f} | perplexity - {train_perplexities[-1]:.3f}\n'
    message += f'Validation: loss - {mean_validation_loss:.4f} | perplexity - {validation_perplexities[-1]:.3f}'
    
    print(message)
    
    #
    #if mean_validation_loss < best_validation_loss:
    #    
    #    best_validation_loss = mean_validation_loss
    #    
    #    torch.save(model.state_dict(), save_best_model_path)
    #    torch.save(optimizer.state_dict(), save_best_optimizer_path)
    #    
    #else:
    #    break
    

Evaluate: 100%|██████████| 5358/5358 [04:03<00:00, 22.00it/s, loss=10.2, perplexity=2.66e+4]


Epoch: 1
Train: loss - 9.7601 | perplexity - 17327.577
Validation: loss - 10.2009 | perplexity - 26927.454


RuntimeError: ignored

In [ ]:
#torch.save(model.state_dict(), save_best_model_path)
#torch.save(optimizer.state_dict(), save_best_optimizer_path)

#torch.save(model.state_dict(), save_last_model_path)
#torch.save(optimizer.state_dict(), save_last_optimizer_path)

In [ ]:
model.load_state_dict(torch.load('./drive/MyDrive/anna/rhyme_last_model_state_dict.pth'))

<All keys matched successfully>

In [ ]:
optimizer.load_state_dict(torch.load('./drive/MyDrive/anna/rhyme_last_optimizer_state_dict.pth'))

In [ ]:
def write_poem(snippet, length, tokenizer, model):
    input_text = f'{snippet}'
    inds = tokenizer(input_text, return_tensors='pt')['input_ids'].to(device)
    text = model.generate(
        input_ids=inds,
        max_length=length + len(inds),
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.9,
        no_repeat_ngram_size=20,
        pad_token_id = 50258
        )
    poem = tokenizer.decode(text[0])
    #poem = poem.strip(f'')
    return poem

In [38]:
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50261, 1536)
    (wpe): Embedding(2048, 1536)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
     

In [41]:
print(raw[4])

ни ша̀тко ни ва̀лко гудѝт кофева̀рка,
гудѝ, виртуо̀зка, гудѝ.
поту̀хли огнѝ моего̀ фейервѐрка,
одна̀ только ко̀поть в грудѝ.
така̀я там ко̀поть, что хо̀чется пла̀кать.
ах, э̀то не прѝхоть моя̀!
моя̀ комсомо̀лка присла̀ла на па̀мять
косы̀нку краснѐе огня̀.
присла̀ла косы̀нку, а что̀ мне в ней то̀лку,
когда̀ только ко̀поть в грудѝ?
вертѝсь же быстрѐе, моя̀ кофемо̀лка,
гудѝ, умоля̀ю, гудѝ!

майо̀р нурутдѝнов кого̀-то свирѐпо
оклѝкнул и вы̀шел на свя̀зь.
смешна̀я дрезѝна включѝла сирѐну
и мѝмо кусто̀в пронесла̀сь.
кляну̀сь, я там бы̀л и смотрѐл дерзновѐнно
сквозь лѝстьев и стра̀хов сквозь слё̀з,
как мѝмо, терза̀я цвету̀щие вѐтви,
возмѐздие вда̀ль унесло̀сь.
дрезѝна взревѐла и вско̀ре умо̀лкла,
утѝхли и слѐзы моѝ.
танцу̀й же быстрѐе, моя̀ виртуо̀зка,
и пла̀менем кра̀сным горѝ

май 




In [42]:
poem = write_poem('ни ша̀тко ни ва̀лко гудѝт кофева̀рка\r\n', 50, tokenizer, model)
print(poem)

ни ша̀тко ни ва̀лко гудѝт кофева̀рка

В этот день в Советском Союзе прошли выборы в Верховный совет СССР. В избирательный бюллетень были включены две фамилии, и от каждой из них исходили


In [43]:
poem = write_poem('ни ша̀тко ни ва̀лко гудѝт кофева̀рка', 100, tokenizer, model)
print(poem)

ни ша̀тко ни ва̀лко гудѝт кофева̀рка̀...

— Ва̀лко, да̀мба̀л! — на̀кричал̀ я.

— Бэ̀л, да̀мба̀л! — повторил̀ он.

— С̀амба̀л? — закричал̀ я.

— Та̀л!

— Кака̀?



In [44]:
poem = write_poem('ни шатко ни валко гудит кофеварка', 100, tokenizer, model)
print(poem)

ни шатко ни валко гудит кофеварка. Как можно было бы все испортить, если бы не эта дурочка, за которой я приехал на такси и которую мне приходится, вопреки моим ожиданиям, ждать до семи утра.

— Пожалуй, я лучше пойду, — говорит она, — мне еще надо приготовить кое-что.

— Конечно, конечно, — говорю я, — но, пожалуй, я тоже пойду.

— Нет


In [46]:
print(raw[100])

пионерские пруды

москва̀ о ско̀лько в э̀том ква̀ке
лягу̀шечьѐй игры̀ и ла̀сковы̀х имѐн
строѐнья со̀ловьѝныѐ бара̀ки
аква̀риу̀м иллю̀зио̀н

заря̀ над пѝонѐрскимѝ пруда̀ми
как бы̀ и впра̀вду пѐрвая̀ заря̀
над мѝром о̀бновлѐнным, со̀ следа̀ми
от па̀триа̀рха ѝ царя̀

но стѐртымѝ бледнѐющѝми в го̀рнем
арха̀нгельско̀м сия̀ниѝ фанфа̀р
и вѐрится̀ что са̀м ты вы̀рван с ко̀рнем
подбро̀шен ввѐрх как пѐрвома̀йский ша̀р

и про̀плыва̀ешь на̀д своѐй столѝцей
бескры̀лой кру̀глой тѐнью пѐрвоптѝцы




In [47]:
poem = write_poem('москва̀ о ско̀лько в э̀том ква̀ке', 50, tokenizer, model)
print(poem)

москва̀ о ско̀лько в э̀том ква̀кѐ? 

Уговорили! Устроили! 

- А вы знаете, - говорит мать, - к нам приезжает товарищ из Москвы. 

- Товарищ


In [48]:
poem2 = write_poem('с днём рождения поздравляю\r\n', 50, tokenizer, model)
print(poem2)

с днём рождения поздравляю
С днём рождения!!!!<s>
Натюрморт с изображением женщины. С днём рождения поздравляю! Счастья, любви, здоровья, успехов, благополучия!
спасибо большое, только вот... женщина за столом - это же


In [49]:
text7 = f'россия великая наша держава\r\n'

In [50]:
input_ids7 = tokenizer(text7, return_tensors='pt')['input_ids'].to(device)
input_ids7

tensor([[ 1584,   274, 28652,  5571,  4985,   342,   206,   203]],
       device='cuda:0')

In [51]:
out7 = model.generate(
    input_ids=input_ids7,
    max_length=max_len + len(input_ids7),
    do_sample=True,
    no_repeat_ngram_size=20,
    pad_token_id = 50258
    )

In [52]:
out7

tensor([[ 1584,   274, 28652,  5571,  4985,   342,   206,   203,   203,   732,
          1858,    16,  3226,    16,  3226,   349, 16490,  7023, 14704,  1642,
            16,  1642,   723,   203,  1140,   436, 31370,    18,   889,   322,
           309, 33015, 21566,    18,   203,  3693, 23136,  3226,  7023, 14704,
          1642,    16,  1642,   723,     1,   203,   672,   345,   613,   727,
           337,  3646,   376,  2197,   411,  3322, 11241, 36800, 22544,  1437,
           294,  3798,  5123,   458,  1182,   337,  3646,     6, 12702,  7975,
         14241, 11088, 32384, 32460,    18,   203,  4441,  2361,   411, 27132,
         16471,    16,  2486,   519,  4319,  1326, 12256,   282,  5806,  9578,
         16864,    18,   203,   677,  1335,   575,  8372,   656,  3984, 13880,
           289]], device='cuda:0')

In [53]:
generated_text7 = tokenizer.decode(out7[0])
generated_text7

'россия великая наша держава\r\n\nИ снова, опять, опять за окном мелькают дома, дома...\nУспокойся. Это не на прогулку идем.\nЗа окнами опять мелькают дома, дома...<s>\nПтица Феникс - одна из самых известных литературных иллюстраций к роману "Феникс" великого русского поэта Сергея Есенина.\nЭто одно из немногих произведений, которое до сих пор живо в памяти современных читателей.\nВ этой же книге мы будем рассматривать и'

In [54]:
print(generated_text7)

россия великая наша держава

И снова, опять, опять за окном мелькают дома, дома...
Успокойся. Это не на прогулку идем.
За окнами опять мелькают дома, дома...<s>
Птица Феникс - одна из самых известных литературных иллюстраций к роману "Феникс" великого русского поэта Сергея Есенина.
Это одно из немногих произведений, которое до сих пор живо в памяти современных читателей.
В этой же книге мы будем рассматривать и
